In [1]:
import json
import re
import datetime
import os

USER_DB = "users.json"
TASK_DB = "tasks.json"

class Storage:
    @staticmethod
    def load(file):
        if not os.path.exists(file):
            return []
        with open(file, "r") as f:
            return json.load(f)

    @staticmethod
    def save(file, data):
        with open(file, "w") as f:
            json.dump(data, f, indent=4)

class User:
    def __init__(self, email, first_name, last_name, phone, password):
        self.email = email
        self.first_name = first_name
        self.last_name = last_name
        self.phone = phone
        self.password = password

    @staticmethod
    def register():
        users = Storage.load(USER_DB)

        first_name = input("Enter your first name: ").strip()
        last_name = input("Enter your last name: ").strip()

        email = input("Enter your email: ").strip()
        if not re.match(r'^[\w\.-]+@[\w\.-]+\.\w+$', email):
            raise ValueError("Invalid email format.")
        if any(user["email"] == email for user in users):
            print("Email already exists. Try logging in.")
            return None

        phone = "01" + input("Enter your phone number (without 01): ").strip()
        if not phone.isdigit() or len(phone) != 11:
            raise ValueError("Phone number must be exactly 11 digits and contain only numbers.")

        password = input("Enter password: ")
        confirm_password = input("Confirm password: ")

        if password != confirm_password:
            print("Passwords do not match!")
            return None

        new_user = {
            "first_name": first_name,
            "last_name": last_name,
            "email": email,
            "phone": phone,
            "password": password
        }

        users.append(new_user)
        Storage.save(USER_DB, users)
        print("Registration successful! You can now log in.")
        return User(**new_user)

    @staticmethod
    def login():
        users = Storage.load(USER_DB)
        email = input("Enter your email: ").strip()
        password = input("Enter your password: ").strip()

        for user in users:
            if user["email"] == email and user["password"] == password:
                print(f"Login successful! Welcome {user['first_name']} {user['last_name']}.")
                return User(**user)
        print("Invalid email or password!")
        return None

class Task:
    def __init__(self, title, description, priority, status, due_date, owner):
        self.title = title
        self.description = description
        self.priority = priority
        self.status = status
        self.due_date = due_date
        self.owner = owner

    def to_dict(self):
        return self.__dict__

    @staticmethod
    def validate_due_date(date_str):
        try:
            due = datetime.datetime.strptime(date_str, "%Y-%m-%d")
            return due > datetime.datetime.now()
        except ValueError:
            return False

class TaskManager:
    def __init__(self, user):
        self.user = user
        self.tasks = Storage.load(TASK_DB)

    def save_tasks(self):
        Storage.save(TASK_DB, self.tasks)

    def user_tasks(self):
        return [t for t in self.tasks if t["owner"] == self.user.email]

    def create_task(self):
        title = input("Enter task title: ").strip()
        description = input("Enter task description (optional): ").strip()
        priority_levels = ["Low", "Medium", "High"]

        while True:
            priority = input("Enter priority (Low, Medium, High): ").strip().capitalize()
            if priority in priority_levels:
                break
            print("Invalid priority! Choose from Low, Medium, or High.")

        due_date = input("Enter due date (YYYY-MM-DD): ").strip()
        if not Task.validate_due_date(due_date):
            print("Invalid due date! Must be in the future.")
            return

        task = Task(title, description, priority, "To-Do", due_date, self.user.email)
        self.tasks.append(task.to_dict())
        self.save_tasks()
        print("Task added successfully!")

    def view_tasks(self):
        user_tasks = self.user_tasks()
        if not user_tasks:
            print("No tasks found!")
            return

        print("\nYour Tasks:")
        for i, task in enumerate(user_tasks, start=1):
            print(f"{i}. {task['title']} - {task['status']} - Due: {task['due_date']}")

    def edit_task(self):
        user_tasks = self.user_tasks()
        if not user_tasks:
            print("No tasks to edit.")
            return

        self.view_tasks()
        try:
            index = int(input("Enter task number to edit: ")) - 1
            if index < 0 or index >= len(user_tasks):
                print("Invalid choice.")
                return

            task = user_tasks[index]
            task["title"] = input("New title (leave empty to keep current): ") or task["title"]
            task["description"] = input("New description (leave empty to keep current): ") or task["description"]
            task["priority"] = input("New priority (Low, Medium, High, leave empty to keep current): ") or task["priority"]
            task["due_date"] = input("New due date (YYYY-MM-DD, leave empty to keep current): ") or task["due_date"]
            task["status"] = input("New status (To-Do, In Progress, Completed): ") or task["status"]

            self.save_tasks()
            print("Task updated successfully!")
        except ValueError:
            print("Invalid input.")

    def delete_task(self):
        user_tasks = self.user_tasks()
        if not user_tasks:
            print("No tasks to delete.")
            return

        self.view_tasks()
        try:
            index = int(input("Enter task number to delete: ")) - 1
            if index < 0 or index >= len(user_tasks):
                print("Invalid choice.")
                return

            self.tasks.remove(user_tasks[index])
            self.save_tasks()
            print("Task deleted successfully!")
        except ValueError:
            print("Invalid input.")

    def search_task(self):
        search_query = input("Enter task title to search: ").strip().lower()
        results = [t for t in self.user_tasks() if search_query in t["title"].lower()]
        if not results:
            print("No matching tasks found.")
            return

        print("\nSearch Results:")
        for task in results:
            print(f"- {task['title']} (Due: {task['due_date']})")

class TaskApp:
    def run(self):
        while True:
            print("\n1. Register\n2. Login\n3. Exit")
            choice = input("Select an option: ").strip()

            if choice == "1":
                User.register()
            elif choice == "2":
                user = User.login()
                if user:
                    manager = TaskManager(user)
                    while True:
                        print("\n1. Create Task\n2. View Tasks\n3. Edit Task\n4. Delete Task\n5. Search Task\n6. Logout")
                        task_choice = input("Choose an option: ").strip()

                        if task_choice == "1":
                            manager.create_task()
                        elif task_choice == "2":
                            manager.view_tasks()
                        elif task_choice == "3":
                            manager.edit_task()
                        elif task_choice == "4":
                            manager.delete_task()
                        elif task_choice == "5":
                            manager.search_task()
                        elif task_choice == "6":
                            break
            elif choice == "3":
                print("Exiting!")
                break

if __name__ == "__main__":
    app = TaskApp()
    app.run()



1. Register
2. Login
3. Exit
Select an option: 1
Enter your first name: hassan
Enter your last name: hussien
Enter your email: has@s.c
Email already exists. Try logging in.

1. Register
2. Login
3. Exit
Select an option: 1
Enter your first name: has
Enter your last name: hu
Enter your email: hu@hu.c
Enter your phone number (without 01): 120194771
Enter password: 123456
Confirm password: 123456
Registration successful! You can now log in.

1. Register
2. Login
3. Exit
Select an option: 2
Enter your email: hu@hu.c
Enter your password: 123456
Login successful! Welcome has hu.

1. Create Task
2. View Tasks
3. Edit Task
4. Delete Task
5. Search Task
6. Logout
Choose an option: 1
Enter task title: has
Enter task description (optional): sss
Enter priority (Low, Medium, High): low
Enter due date (YYYY-MM-DD): 2025-09-08
Task added successfully!

1. Create Task
2. View Tasks
3. Edit Task
4. Delete Task
5. Search Task
6. Logout
Choose an option: 2

Your Tasks:
1. has - To-Do - Due: 2025-09-08

